# Agents evaluators

Agents emit messages, and providing the above inputs typically require parsing messages and extracting the relevant information. If you're building agents using Azure AI Agent Service, we provide native integration for evaluation that directly takes their agent messages.

> https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators/agent-evaluators

In [1]:
import datetime
import os
import sys

from azure.ai.evaluation import AzureOpenAIModelConfiguration, IntentResolutionEvaluator, TaskAdherenceEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from pprint import pprint

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 26-Jun-2025 12:28:43


In [4]:
load_dotenv("azure.env")

endpoint = os.getenv("endpoint")
key = os.getenv("key")

In [5]:
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=endpoint,
    api_key=key,
    azure_deployment="gpt-4.1",
    api_version="2024-10-21",
)

## Intent resolution

> Measures how accurately the agent identifies and addresses user intentions.

IntentResolutionEvaluator measures how well the system identifies and understands a user's request, including how well it scopes the user’s intent, asks clarifying questions, and reminds end users of its scope of capabilities. Higher score means better identification of user intent.

In [6]:
intent_resolution = IntentResolutionEvaluator(model_config=model_config, threshold=3)

Class IntentResolutionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [7]:
response = intent_resolution(
    query="What are the opening hours of the Eiffel tower?",
    response="Opening hours of the Eiffel tower are 9:00 AM to 11:00 PM.")

pprint(response, width=150)

{'additional_details': {'actual_user_intent': 'find out the opening hours of the Eiffel Tower',
                        'agent_perceived_intent': 'provide the opening hours of the Eiffel Tower',
                        'conversation_has_intent': True,
                        'correct_intent_detected': True,
                        'intent_resolved': False},
 'intent_resolution': 4.0,
 'intent_resolution_reason': "The response directly answers the user's question by providing the opening hours of the Eiffel Tower. However, it "
                             'lacks a disclaimer about possible seasonal changes or special dates, and does not specify if these hours are current '
                             'or may vary. While the answer is clear and relevant, it could be improved with a note about checking for updates or '
                             'exceptions. Thus, it addresses the intent with moderate accuracy but has minor omissions.',
 'intent_resolution_result': 'pass',
 'intent_re

In [8]:
response = intent_resolution(
    query="What are the opening hours of the Eiffel tower?",
    response="Closing hours of the Montparnasse tower are 9:00 AM to 8:00 PM."
)

pprint(response, width=150)

{'additional_details': {'actual_user_intent': 'find out the opening hours of the Eiffel Tower',
                        'agent_perceived_intent': 'provide closing hours for Montparnasse Tower',
                        'conversation_has_intent': True,
                        'correct_intent_detected': False,
                        'intent_resolved': False},
 'intent_resolution': 1.0,
 'intent_resolution_reason': "The agent's response is completely unrelated to the user's query. The user asked for the opening hours of the Eiffel "
                             'Tower, but the response provides the closing hours for the Montparnasse Tower, which is a different landmark and does '
                             "not address the user's request in any way.",
 'intent_resolution_result': 'fail',
 'intent_resolution_threshold': 3}


### Task adherence

> Measures how well the agent follows through on identified tasks.

In various task-oriented AI systems such as agentic systems, it's important to assess whether the agent has stayed on track to complete a given task instead of making inefficient or out-of-scope steps. TaskAdherenceEvaluator measures how well an agent’s response adheres to their assigned tasks, according to their task instruction (extracted from system message and user query), and available tools. Higher score means better adherence of the system instruction to resolve the given task.

In [9]:
task_adherence = TaskAdherenceEvaluator(model_config=model_config, threshold=3)

Class TaskAdherenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [10]:
resp = task_adherence(
    query="What are the best practices for maintaining a healthy rose garden during the summer?",
    response="Make sure to water your roses.")

pprint(resp, width=150)

{'task_adherence': 2.0,
 'task_adherence_reason': 'The response gives one relevant but extremely limited suggestion, missing most of the expected content for "best '
                          'practices."',
 'task_adherence_result': 'fail',
 'task_adherence_threshold': 3}


In [11]:
resp = task_adherence(
    query="What are the best practices for maintaining a healthy rose garden during the summer?",
    response="""
        Water deeply 2–3 times per week, preferably in the morning.
Mulch around plants to retain moisture and suppress weeds.
Deadhead spent blooms and lightly prune for air circulation.
Monitor for pests and diseases; treat early with natural remedies.
Fertilize every 4–6 weeks with a balanced or high-potassium fertilizer.
Keep the garden clean by removing fallen leaves and petals.
Check soil health and amend as needed for optimal growth.""")

pprint(resp, width=150)

{'task_adherence': 5.0,
 'task_adherence_reason': 'The response is comprehensive, accurate, and directly follows the instructions, providing a complete set of best '
                          'practices for summer rose garden care.',
 'task_adherence_result': 'pass',
 'task_adherence_threshold': 3}


In [12]:
resp = task_adherence(
    query="What are the best practices for maintaining a healthy rose garden during the summer?",
    response="Buy some new flowers")

pprint(resp, width=150)

{'task_adherence': 1.0,
 'task_adherence_reason': 'The response is entirely irrelevant to the question and does not provide any useful or related information about rose '
                          'garden maintenance.',
 'task_adherence_result': 'fail',
 'task_adherence_threshold': 3}
